https://ratsgo.github.io/deep%20learning/2017/05/13/GRU/
https://aikorea.org/blog/rnn-tutorial-4/

[오차 역전파](https://ratsgo.github.io/deep%20learning/2017/05/14/backprop/)

# Gated Recurrent Unit(GRU)

## GRU 개요

GRU는 LSTM의 장점을 유지하면서도 계산복잡성을 낮춘 셀 구조이다.  
GRU도 Gradient Vanishing/Explosing 문제를 극복했다는 점에서 LSTM과 유사하지만 LSTM 보다 일부의 **Gate**를 생략한 형태이다.  
GRU는 크게 **update gate**와 **rest gate**두 가지로 나눠진다.  
  
두 게이트 모두 현 시점의 입력값($x_t$)과 직전 시점 은닉층값($h_{t-1}$)을 반영해 값을 구한다.  
**활성함수**($\sigma$)는 시그모이드를 사용한다.  
  
$$
update\;gate\;\;\;:\;\;\;z_t = \sigma(W^{(z)} x_t + U^{(z)} h_{t-1})\\
reset\;gate\;\;\;:\;\;\;r_t = \sigma(W^{(r)}x_t + U^{(r)}h_{t-1})
$$

$W, U$는 각각 입력값과 은닉층값이 선형결합하는 파라미터이다.  
위 첨자 $z, r$은 각각 **update gate**, **reset gate**에 속한다는 뜻이다.  

----

메모리(기억)는 현 시점($t$)에서 기억해둘 만한 정보를 아래와 같이 정의한다.  
$$
\tilde{h_t} = tanh(W x_t + r_t \odot U h_{h-1})
$$
* $\tilde{A}$는 A의 중앙값을 의미 그러나 위 식에서는 어떤 값을 말하는지 해석되지 않음
* $\odot$은 Hadamard product(요소별 곱셈, 내적?)을 뜻한다.

위 식을 해석하면 현시점 정보($Wx_t$)와 과거정보($Uh_{t-1}$)를 반영하되,  
과거정보를 얼마나 반영할지는 **reset gate**($r_t$)값에 의존한다는 이야기다.

**reset gate**의  활섬함수는 시그모이드 이므로 0~1사이의 범위를 갖는다.  
> $sigmoid()$ activation function
* 0~1 사이의 범위를 갖는다.  
  
> $r_t$값이   
* 0이라면 과거 정보를 모두 잊고, 
* 1이라면 과거 정보를 모두 기억한다.  
  
$r_t$값에 상관없이 현재 정보는 반영된다.  

위 식의 활성함수($tanh$)는 하이퍼볼릭탄젠트이다.  
> $tanh()$  activation function  
* -1~1사이의 범위를 갖는다.  
  
위 식엔 $W, U$에 위 첨자가 없는데, 아예 다른 파라메터라는 점을 주의해야한다.  

---

다음 상태(state)로 업데이트 하는 식은 아래와 같다.
$$
h_t = z_t \odot h_{t-1} + (1-z_t) \odot \tilde{h_t}
$$

위의 식을 해석하면 아래와 같다.  
* $h_{t-1}$는 과거 정보, 
* $\tilde{h_t}$(틸다)는 현재 정보이다.  
이를 얼마나 조합할지 결정하는 것은 **update gate**의 $z_t$이다.  

**update gate**의 함수 또한 시그모이드이므로 0~1사이의 범위값을 가진다. $z_t = \{x:0<x<1\}$  
* $z_t$가 0이라면 과거 정보를 모두 잊고, 현재 정보만을 기억한다.  
* $z_t$가 1이라면 과거 정보를 모두 기억하지만, 현재정보를 모두 무시한다.  


GRU 셀을 그림([출처](http://i.imgur.com/rehjrBZ.png))으로 나타내면 아래와 같다.  
![](http://i.imgur.com/rehjrBZ.png)

---

## GRU의 순전파
[순전파, 역전파에 관한 참고 글](https://ratsgo.github.io/deep%20learning/2017/05/14/backprop/)

GRU의 순전파를 계산 그래프로 나타내면 아래 그림과 같다.  
그림으로 보면 복잡해 보이나 위에서 설명한 수식과 동일하다.  
다만 $h_t$에 $W_{out}$을 곱하여 $y_t$를 만드는 과정은 엄밀하게 GRU셀 내부 작동은 아니지만 이해를 돕기 위해 그림에 포함되었다.  
![](http://i.imgur.com/nenqTrN.png)

$t$ 시점의 GRU 셀의 입력은 $x_t$, $h_{t-1}$, 출력은 $h_t$이다.  
GRU셀과 연결된 Softmax-with-Loss 계층은 y_t를 입력으로 받아 $t$시점의 loss $L_t$를 출력한다.  
  
우리가 확인하고 업데이트해야할 파라미터는 $U^z, U^r, U, W^z, W^r, W$이다.

---

## GRU의 역전파

$t$시점의 GRU셀이 Softmax-with-Loss노드로부터 최초로 받은 그래디언트는 $\partial L_t/ \partial y_t$이다.  
아래 그림에서는 이를 편의상 $d y_t$라고 사용한다.  
이후 모든 표기는 이 방식을 따른다.  
$h_t$와 $y_t$는 곱셈 노드로 연결되어 있기 때문에 $dW_{out}$과 $dh_t$는 흘러들어온 그래디언트 $dy_t$에 순전파 때 입력신호들을 서로 바꾼 값을 각각 곱한 값이다.

$\tilde{dh_t}$(틸다)는 흘러들어온 그래디언트 $dh_t$에 $\odot$(Hadamard product)의 로컬 그래디언트를 곱해 구한다.  

<br>
우선 그림의 위쪽을 살펴본다.  

* $\tilde{dr_t}$(틸드)는 흘러들어온 그래디언트 $\tilde{dh_t}$(틸드)에 하이퍼볼릭탄젠트로 로컬 그래디언트를 곱해 구한다.  

* $dr_t$는 흘러들어온 그래디언트 $\tilde{dr_t}$(틸드)에 $\odot$(Hadamard product)의 로컬 그래디언트를 곱해 구한다. 

* $dinput_r$은 흘러들어온 그래디언트 $dr_t$에 시그모이드 함수의 로컬 그래디언트를 곱해 구한다. 

* $dUh_{t-1}$은 흘러들어온 그래디언트 $\tilde{dr_t}$(틸드)에 $\odot$(Hadamard product)의 로컬 그래디언트를 곱해 구한다.

![](https://i.imgur.com/SLhyNgK.png)

이제 그림 아래쪽을 살펴본다.
* $dz_t$는 흘러들어온 그래디언트 $dh_t$에 $\odot$(Hadamard product)의 로컬 그래디언트를 곱해서 구한다. 

* $dinput_z$는 흘러들어온 그래디언트 $dz_t$에 시그모이드 함수의 로컬 그래디언트를 곱해서 구한다.  

* $dWx_t$는 흘러들어온 그래디언트 $\tilde{dh_t}$에 하이퍼볼릭탄젠트의 로컬 그래디언트를 곱해 구한다.  

* 위쪽의 $input_r$과 아래쪽의 $input_r$은 덧셈 노드로 연결되어 있으므로 흘러들어온 그래디언트가 그대로 전파된다.  

위 상황을 종합하면 우리가 구해야 하는 $dW_z$와 $dU_z$는 아래와 같다.  
$dh_t$는 $\partial L_t/\partial h_t$를 의미한다.

> $cvx$를 기준으로 그림의 위쪽은 모두 $\partial h_t / \partial U_t$를 계산하는 과정이라고 라고 이해하면 된다.  
$$
\frac{\partial L_t}{\partial U_x} = 
[dinput_z \bullet h_{t-1}\;\;dinput_r \bullet h_{t-1}\;\;dWh_{t-1}\bullet h_{t-1}] = \\
\frac{\partial L_t}{\partial y_t} \frac{\partial y_t}{\partial h_t} \frac{\partial h_t}{\partial U_h} = \frac{\partial L_t}{\partial h_t} \frac{\partial h_t}{\partial U_h}
$$
  
> 마찬가지로   
$cvx$를 기준으로 그림의 아래쪽은 모두 $\partial h_t/\partial W_x$를 계산하는 과정으로 보면 된다.
$$
\frac{\partial L_t}{\partial W_x} = [dinput_z \bullet x_t\;\;dinput_r \bullet x_t\;\;dWx_t\bullet x_t] = \\
\frac{\partial L_t}{\partial y_t} \frac{\partial y_t}{\partial h_t} \frac{\partial h_t}{\partial W_x} = \frac{\partial L_t}{\partial h_t} \frac{\partial h_t}{\partial W_x}
$$

## The BackPropagation Through Time(BPTT)

지금까지 설명한 그림은 $t$ 시점의 셀하나였다.  
그런데 셀을 다양하게 구성해 RNN네크워크를 구성할 수 있다.  
GRU 역전파시 그래디언트는 아래 그림의 파란색 셀 개수만큼 재귀적으로 합쳐져 전파된다.  
  
![](http://i.imgur.com/Q8zv6TQ.png)

다시 GRU의 t시점의 셀로 간다.  
$\delta_t$는 흘러들어온 그래디언트$dh_t(=\partial L_t/ \partial h_t)$에 Hadamard product의 로컬 그래디언트 $\partial h_t/ \partial h_{t-1}$를 곱해 구한다.  
이를 수식으로 정리하면 아래와 같다.  


$$
\delta_t = \frac{\partial L_t}{\partial y_t} \frac{\partial y_t}{\partial h_t} \frac{\partial h_t}{\partial h_{t-1}}
$$

위 식에서 $h_{t-1}$은 $t-1$시점의 GRU셀의 출력 결과라는 점에 주목해야 한다.  
다시 말해 $\delta_t$는 적색 화살표를 따라 $t-1$시점의 GRU셀로 흘러들어간다.  
  
![](http://i.imgur.com/2QzVDbi.png)

이번엔 $t-1$시점의 GRU 셀을 